In [1]:
import numpy as np
import pandas as pd
import numpy as np
import os
import time
from google.cloud import bigquery
import re

In [2]:
os.chdir('G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Dados para subir\\Tabelas finais')


In [39]:
df = pd.read_csv('SIAPE_ativos_federal_raca_sexo_orgao_superior.csv')
df

,ano,orgao_superior,raca,genero,total_vinculos
0,2020,CGU,Branca,Fem,446
1,2020,CGU,NÃ£o informada,Fem,25
2,2020,CGU,Negra,Fem,171
3,2020,CGU,Outros,Fem,17
4,2020,MAPA,Branca,Fem,1986
...,...,...,...,...,...
131,2020,MTUR,Outros,Mas,22
132,2020,PR,Branca,Mas,3381
133,2020,PR,NÃ£o informada,Mas,163
134,2020,PR,Negra,Mas,1217


In [40]:
df['raca'].unique()

array(['Branca', 'NÃ£o informada', 'Negra', 'Outros'], dtype=object)

In [41]:
df['genero'].unique()

array(['Fem', 'Mas'], dtype=object)

In [42]:
df = df.rename(columns={"raca" :"cor_raca", "total_vinculos" : "quantidade_vinculos"})
df

,ano,orgao_superior,cor_raca,genero,quantidade_vinculos
0,2020,CGU,Branca,Fem,446
1,2020,CGU,NÃ£o informada,Fem,25
2,2020,CGU,Negra,Fem,171
3,2020,CGU,Outros,Fem,17
4,2020,MAPA,Branca,Fem,1986
...,...,...,...,...,...
131,2020,MTUR,Outros,Mas,22
132,2020,PR,Branca,Mas,3381
133,2020,PR,NÃ£o informada,Mas,163
134,2020,PR,Negra,Mas,1217


In [43]:
def transformar(nome):
    nome = re.sub(r"\bFem\b", "Mulheres", nome)
    nome = re.sub(r"\bMas\b", "Homens", nome)
    nome = re.sub(r"\bNÃ£o informada\b", "Não Informado", nome)
    return nome

In [44]:
df['cor_raca'] = df['cor_raca'].apply(transformar)
df['genero'] = df['genero'].apply(transformar)

In [45]:
df['genero_raca'] = df['genero'] + ' ' + df['cor_raca']

In [47]:
def transforme(nome):
    nome = re.sub(r"\bMulheres Branca\b", "Mulheres Brancas", nome)
    nome = re.sub(r"\bMulheres Não Informado\b", "Mulheres sem informação de raça", nome)
    nome = re.sub(r"\bMulheres Negra\b", "Mulheres Negras", nome)
    nome = re.sub(r"\bMulheres Outros\b", "Mulheres com outras raças", nome)
    nome = re.sub(r"\bHomens Branca\b", "Homens Brancos", nome)
    nome = re.sub(r"\bHomens Não Informado\b", "Homens sem informação de raça", nome)
    nome = re.sub(r"\bHomens Negra\b", "Homens Negros", nome)
    nome = re.sub(r"\bHomens Outros\b", "Homens com outras raças", nome)
    return nome

In [50]:
df['genero_raca'] = df['genero_raca'].apply(transforme)

In [52]:
df['genero_raca'].unique()

array(['Mulheres Brancas', 'Mulheres sem informação de raça',
       'Mulheres Negras', 'Mulheres com outras raças', 'Homens Brancos',
       'Homens sem informação de raça', 'Homens Negros',
       'Homens com outras raças'], dtype=object)

In [53]:
df = df[['ano', 'orgao_superior', 'genero', 'cor_raca', 'genero_raca', 'quantidade_vinculos']]

In [54]:
df

,ano,orgao_superior,genero,cor_raca,genero_raca,quantidade_vinculos
0,2020,CGU,Mulheres,Branca,Mulheres Brancas,446
1,2020,CGU,Mulheres,Não Informado,Mulheres sem informação de raça,25
2,2020,CGU,Mulheres,Negra,Mulheres Negras,171
3,2020,CGU,Mulheres,Outros,Mulheres com outras raças,17
4,2020,MAPA,Mulheres,Branca,Mulheres Brancas,1986
...,...,...,...,...,...,...
131,2020,MTUR,Homens,Outros,Homens com outras raças,22
132,2020,PR,Homens,Branca,Homens Brancos,3381
133,2020,PR,Homens,Não Informado,Homens sem informação de raça,163
134,2020,PR,Homens,Negra,Homens Negros,1217


Subindo para o GBQ

In [55]:
client = bigquery.Client()
dataset_ref = client.dataset('perfil_remuneracao')

c:\Users\ana.sales_republica\Anaconda\lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [56]:
schema = [bigquery.SchemaField('ano', 'INTEGER', description= 'Ano de referência da observação'),
          bigquery.SchemaField('orgao_superior', 'STRING', description= 'Orgão superior (Controladoria-Geral da União (CGU), Ministérios) observado'),
          bigquery.SchemaField('genero', 'STRING', description= 'Gênero autodeclarado ou não'),
          bigquery.SchemaField('cor_raca', 'STRING', description= 'Raça/cor da pessoa observada'),
          bigquery.SchemaField('genero_raca', 'STRING', description= 'Gênero e raça da pessoa observada'),
          bigquery.SchemaField('quantidade_vinculos', 'INTEGER', description= 'Número total de vinculos observados')]

In [57]:
table_ref = dataset_ref.table('SIAPE_ativos_federal_raca_sexo_orgao_superior')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result() 


LoadJob<project=repositoriodedadosgpsp, location=US, id=5befb63c-685c-4d66-a0e8-f8b6e7cccc0b>